In [13]:
!pip install groq

In [ ]:
import os

from groq import Groq
client = Groq(
    api_key="--",
)

In [15]:
def get_completion(prompt, model="openai/gpt-oss-20b"):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
        temperature=0,
    )
    return chat_completion.choices[0].message.content

In [16]:
review = """
Wanted a smartwatch mainly for fitness tracking, and this \
one looked like a good balance of features and price. \
Shipping was quick — arrived in just two days and the box \
was neatly packed. Setup was super easy, just a few taps \
to pair it with my phone. The heart rate and step tracking \
seem accurate, and I really like the reminders to stand up \
and move around when I’ve been sitting too long. The first \
strap I received had a loose clasp, so I reached out to \
support and they immediately sent me a replacement, which \
arrived within a few days. The screen is bright and easy \
to read outdoors, and the battery lasts me about five days \
on a single charge. I do wish it had more customization \
for workouts, but overall I’m really happy with it. FitX \
seems like a solid company that actually listens to its \
customers!
"""


In [17]:
#Sentiment (positive/negative)

prompt = f"""
What is the sentiment of the following product review,
which is delimited with triple backticks?

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

**Sentiment:** Positive.


In [18]:
prompt = f"""
What is the sentiment of the following product review,
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

positive


In [19]:
# Identify types of emotions

prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

happy, satisfied, relieved, grateful, appreciative


In [20]:
# Identify anger

prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

no


# Extract product and company name from customer reviews

In [21]:
prompt = f"""
Identify the following items from the review text:
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

{"Item":"smartwatch","Brand":"FitX"}


In [23]:
# Doing multiple tasks at once

prompt = f"""
Identify the following items from the review text:
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

{"Sentiment":"positive","Anger":false,"Item":"smartwatch","Brand":"FitX"}


# Inferring topics

In [24]:
story = """
In a recent survey conducted by the government,
public sector employees were asked to rate their level
of satisfaction with the department they work at.
The results revealed that GraceUP was the most popular
organization with a satisfaction rating of 95%.

One GraceUP employee, John Smith, commented on the findings,
stating, "I’m not surprised that GraceUP came out on top.
It’s a fantastic place to work with supportive colleagues
and inspiring projects. I’m proud to be a part of such a
forward-thinking organization."

The results were also welcomed by GraceUP’s management team,
with Director Tom Johnson stating, "We are delighted to see
such positive feedback from our employees. At GraceUP, we
strive to create an environment where our people feel valued
and motivated, and it’s rewarding to know that effort is
making a difference."

The survey also revealed that the
Social Security Administration had the lowest satisfaction
rating, with only 45% of employees indicating they were
satisfied with their job. The government has pledged to
address the concerns raised by employees in the survey and
work towards improving job satisfaction across all departments.
"""


In [26]:
#Infer 5 topics

prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long.

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

GraceUP, employee satisfaction, survey, government, Social Security Administration


In [27]:
response.split(sep=',')

['GraceUP',
 ' employee satisfaction',
 ' survey',
 ' government',
 ' Social Security Administration']

In [28]:
topic_list = [
    "GraceUP", "public sector", "employee satisfaction",
    "government survey", "workplace culture"
]


In [29]:
# Make a news alert for certain topics

prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as follows:
item from the list: 0 or 1

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

GraceUP: 1  
public sector: 1  
employee satisfaction: 1  
government survey: 1  
workplace culture: 1


In [31]:
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}
if topic_dict['GraceUP'] == 1:
    print("ALERT: New GraceUP story!")

ALERT: New GraceUP story!


# Theory about Inferring

In **prompt engineering**, *inferring* usually refers to how an AI model **draws conclusions or extracts meaning** from the input you provide — even if that meaning isn’t stated explicitly.

Here’s the breakdown:

### 🔹 What “inferring” means in this context

* It’s the model’s ability to **connect dots** and **derive hidden intent** behind a prompt.
* Instead of just repeating instructions, the AI interprets **implied goals, context, or constraints**.
* Basically: *“What is the user really asking me to do here?”*

### 🔹 Example

Prompt:

> “Write a short email apologizing for a delivery delay.”

The AI **infers** that:

* The email should be professional and polite.
* The writer is likely representing a company.
* The tone should acknowledge the delay and offer reassurance.

Even though none of that is explicitly spelled out, the model pulls it from context.

### 🔹 Why it matters in prompt engineering

When you design prompts, you can either:

1. **Rely on inference** (let the AI fill in gaps using common sense/context), or
2. **Minimize inference** (spell things out clearly so the AI doesn’t make assumptions).

Balancing this is key: too much inference can cause *hallucinations* or wrong assumptions, but too little can make prompts clunky and over-engineered.

---

⚡ In short: **Inferring in prompt engineering is the model’s way of reading between the lines to understand intent, context, or missing details.**

